In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

# Rheus_macaques_ifgn Cynomolgus_monkeys_ifgn Human_ifgn Human_with_h1n1
adata1 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/Rheus_macaques_ifgn.h5ad') # Please modify this to your address.
adata1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnnData object with n_obs × n_vars = 112218 × 39
    obs: 'sample', 'cluster_o', 'cluster_s'

In [2]:
adata1.obs['cluster_s']

0                  B
1         Neutrophil
2         Neutrophil
3              CD8 T
4              CD4 T
             ...    
112213            NK
112214             B
112215    Neutrophil
112216    Neutrophil
112217    Neutrophil
Name: cluster_s, Length: 112218, dtype: category
Categories (6, object): ['B', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil']

In [3]:
adata1.var_names

Index(['CD235a', 'CD45', 'STAT3', 'CD61', 'CD7', 'CD33', 'CD11c', 'MAPKAPK2',
       'CREB', 'PLCg2', 'STAT1', 'CD123', 'STAT5', 'CD14', 'Erk1.2', 'CD11b',
       'p38', 'CD8', 'CD4', 'CD3', 'CD66', 'CD16', 'Zap70.Syk', 'TBK1', 'CD1c',
       'IkBa', 'BDCA3', 'STAT6', 'CD45RA', 'Fox.P3', 'CD161', 'Ki67', 'STAT4',
       'CCR7', 'X4EBP1', 'CD19', 'IgM', 'CD56', 'HLA.DR'],
      dtype='object')

In [4]:
adata2 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/Human_ifgn.h5ad') # Please modify this to your address.
adata2

AnnData object with n_obs × n_vars = 114175 × 40
    obs: 'sample', 'cluster_o', 'cluster_s'

In [5]:
var_names_union = adata1.var_names.intersection(adata2.var_names)
print(var_names_union)

Index(['CD235a', 'CD45', 'STAT3', 'CD61', 'CD7', 'CD33', 'CD11c', 'MAPKAPK2',
       'CREB', 'PLCg2', 'STAT1', 'CD123', 'STAT5', 'CD14', 'Erk1.2', 'CD11b',
       'p38', 'CD8', 'CD4', 'CD3', 'CD66', 'CD16', 'Zap70.Syk', 'TBK1', 'CD1c',
       'IkBa', 'BDCA3', 'STAT6', 'CD45RA', 'Fox.P3', 'CD161', 'Ki67', 'STAT4',
       'CCR7', 'X4EBP1', 'CD19', 'IgM', 'CD56', 'HLA.DR'],
      dtype='object')


In [6]:
new_index = var_names_union.difference(['index'])
new_index

Index(['BDCA3', 'CCR7', 'CD11b', 'CD11c', 'CD123', 'CD14', 'CD16', 'CD161',
       'CD19', 'CD1c', 'CD235a', 'CD3', 'CD33', 'CD4', 'CD45', 'CD45RA',
       'CD56', 'CD61', 'CD66', 'CD7', 'CD8', 'CREB', 'Erk1.2', 'Fox.P3',
       'HLA.DR', 'IgM', 'IkBa', 'Ki67', 'MAPKAPK2', 'PLCg2', 'STAT1', 'STAT3',
       'STAT4', 'STAT5', 'STAT6', 'TBK1', 'X4EBP1', 'Zap70.Syk', 'p38'],
      dtype='object')

In [7]:
adata1 = adata1[:, new_index]
adata2 = adata2[:, new_index]

In [8]:
celltype = np.intersect1d(adata1.obs['cluster_s'].values, adata2.obs['cluster_s'].values)
celltype

array(['B', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil'], dtype=object)

In [9]:
x1 = adata1.X
y1 = adata1.obs['cluster_s']

x, y = None, None
celltype_num = len(celltype)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x1[np.array(y1 == celltype[i]), :])

for i in range(20000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 200)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
(39, 20000) (6, 20000)


In [10]:
df = pd.DataFrame(x.T, columns=adata1.var_names)
df

,BDCA3,CCR7,CD11b,CD11c,CD123,CD14,CD16,CD161,CD19,CD1c,...,PLCg2,STAT1,STAT3,STAT4,STAT5,STAT6,TBK1,X4EBP1,Zap70.Syk,p38
0,3.308086,3.866609,4.328790,2.230194,1.314075,1.995373,3.134111,3.165114,4.036757,1.301073,...,1.013946,1.916919,3.638864,1.027625,1.533087,0.871742,1.126457,1.560505,0.738924,1.445570
1,2.918744,3.567778,3.494488,1.477665,1.211893,1.774938,2.285348,2.814540,3.845926,1.390819,...,0.863767,1.702716,3.427611,0.989072,1.337087,0.812486,1.098331,1.500010,0.753133,1.400922
2,3.099449,3.657715,3.822262,1.583867,1.199429,1.853734,2.512610,2.939135,3.504078,0.954161,...,0.940701,1.868436,3.461378,0.999885,1.453683,0.794775,1.074420,1.401685,0.768968,1.400074
3,2.722284,3.658065,4.096998,1.412922,1.071342,1.845654,2.512217,2.789737,3.186491,0.769837,...,0.843624,1.650830,3.348369,1.021187,1.446977,0.862114,1.054204,1.314636,0.810827,1.433716
4,3.255347,3.976033,4.530627,1.863358,1.308290,2.056197,2.839639,3.094761,3.793454,1.200152,...,1.065371,2.040483,3.568730,1.188823,1.626341,0.973339,1.162831,1.677036,0.851887,1.538005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3.373937,3.854571,4.223957,2.006768,1.373079,2.057830,2.883171,3.125452,3.821935,1.080848,...,0.985977,2.180135,3.601029,1.013103,1.525209,0.794903,1.108991,1.564339,0.718195,1.466745
19996,2.935168,3.569510,3.660029,1.655573,1.100008,1.765320,2.685171,2.848798,3.666749,1.053327,...,0.921246,1.595367,3.496497,0.947428,1.335072,0.791532,1.012084,1.380866,0.703163,1.345613
19997,3.155396,3.778891,4.504331,1.917033,1.203074,2.055048,2.972636,2.956690,3.569181,1.043002,...,0.991717,1.945313,3.564967,1.061236,1.571587,0.882412,1.052651,1.421386,0.779184,1.437256
19998,3.202277,3.863933,4.645606,1.814016,1.310050,2.155351,2.722515,3.089172,3.712319,1.060074,...,0.975822,1.961029,3.599791,1.120726,1.518334,0.829671,1.079551,1.422632,0.882217,1.448133


In [11]:
adata_0 = sc.AnnData(df)
for i in range(len(celltype)):
    adata_0.obs[celltype[i]] = y[i].T
adata_0.obs['batch'] = np.array(['0'] * adata_0.shape[0])
adata_0.uns['cell_types'] = celltype

adata_0

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 20000 × 39
    obs: 'B', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [12]:
x2 = adata2.X
y2 = adata2.obs['cluster_s']

x, y = None, None
celltype_num = len(celltype)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x2[np.array(y2 == celltype[i]), :])

for i in range(4000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 200)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
1000
2000
3000
(39, 4000) (6, 4000)


In [13]:
df2 = pd.DataFrame(x.T, columns=adata2.var_names)
df2

,BDCA3,CCR7,CD11b,CD11c,CD123,CD14,CD16,CD161,CD19,CD1c,...,PLCg2,STAT1,STAT3,STAT4,STAT5,STAT6,TBK1,X4EBP1,Zap70.Syk,p38
0,1.968154,3.550802,3.234215,1.208096,0.827063,1.768892,2.498796,2.977000,1.148220,0.490890,...,1.027621,1.534868,2.798660,1.194288,1.525057,0.844373,1.212216,1.388869,0.794165,1.671625
1,2.285993,3.707497,3.827792,1.782208,0.923514,2.330458,2.595241,3.111139,1.325197,0.583988,...,1.198040,1.989780,2.946741,1.330300,1.712744,0.920396,1.244342,1.709643,0.825219,1.755012
2,2.054615,3.586604,3.738093,1.673656,0.740746,2.160888,2.809325,3.038413,0.676095,0.431786,...,1.169831,1.776588,2.939265,1.151145,1.579173,0.783658,1.103741,1.289109,0.772887,1.551515
3,1.749355,3.392489,3.477710,1.197348,0.672870,1.442251,3.122654,2.894201,0.749392,0.442846,...,1.029089,1.662512,2.790040,1.042596,1.507647,0.711407,1.032688,1.160498,0.876864,1.453313
4,1.947464,3.589967,3.417028,1.322051,0.767012,1.739216,2.806953,2.949715,0.932581,0.483956,...,1.113858,1.626568,2.992600,1.157477,1.608522,0.837979,1.152914,1.335847,0.790891,1.653215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1.994204,3.594338,3.716000,1.558561,0.804035,1.987689,2.892093,2.928991,1.110977,0.511983,...,1.195415,1.820387,2.964990,1.128459,1.596828,0.790096,1.083485,1.385663,0.841879,1.579757
3996,2.098389,3.657377,2.901783,1.031742,0.816851,1.809796,1.943480,3.032566,1.632853,0.539950,...,1.139255,1.834591,2.792318,1.212769,1.581267,0.893765,1.280680,1.600130,0.806168,1.752390
3997,2.238187,3.702267,4.164973,1.965853,0.911923,2.309295,3.308335,3.091828,1.386075,0.576120,...,1.178818,1.916420,2.921054,1.275184,1.713255,0.949619,1.244314,1.634171,0.849404,1.700239
3998,2.781682,4.002808,3.723809,1.908308,1.096761,2.892183,2.232625,3.359213,2.017759,0.749837,...,1.401871,2.054402,2.911207,1.558323,1.776852,0.998474,1.506015,2.205201,0.879226,1.878038


In [14]:
adata_1 = sc.AnnData(df2)
for i in range(len(celltype)):
    adata_1.obs[celltype[i]] = y[i].T
adata_1.obs['batch'] = np.array(['1'] * adata_1.shape[0])
adata_1.uns['cell_types'] = celltype

adata_1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 4000 × 39
    obs: 'B', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [15]:
adata_0.var_names_make_unique()
adata_1.var_names_make_unique()
adata = sc.concat((adata_0, adata_1))
adata.uns['cell_types'] = celltype
adata

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 24000 × 39
    obs: 'B', 'CD4 T', 'CD8 T', 'MC', 'NK', 'Neutrophil', 'batch'
    uns: 'cell_types'

In [16]:
# Rheus_macaques Cynomolgus_monkeys Human h1n1
adata.write_h5ad('/public/home/syj/scpDeconv/dataset/mixup_data/Rheus_macaques_2_Human.h5ad') # Please modify this to your address.